In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-bills/fake_bills.csv


In [2]:
df = pd.read_csv("/kaggle/input/fake-bills/fake_bills.csv", delimiter=';')
df.isnull().sum()
#Using two different dataframes to see which provides the best model for ML
df_with_mean_input = df.fillna(df.mean())
df_with_mean_input.isnull().sum()
df_dropped = df.dropna()
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1463 entries, 0 to 1499
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   is_genuine    1463 non-null   bool   
 1   diagonal      1463 non-null   float64
 2   height_left   1463 non-null   float64
 3   height_right  1463 non-null   float64
 4   margin_low    1463 non-null   float64
 5   margin_up     1463 non-null   float64
 6   length        1463 non-null   float64
dtypes: bool(1), float64(6)
memory usage: 81.4 KB


In [3]:
# Preparing data for ML
y_mean = df_with_mean_input['is_genuine']
#print(y_mean)
X_mean = df_with_mean_input.drop(columns = 'is_genuine')
X_mean.info()

y_dropped = df_dropped['is_genuine']
X_dropped = df_dropped.drop(columns = 'is_genuine')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   diagonal      1500 non-null   float64
 1   height_left   1500 non-null   float64
 2   height_right  1500 non-null   float64
 3   margin_low    1500 non-null   float64
 4   margin_up     1500 non-null   float64
 5   length        1500 non-null   float64
dtypes: float64(6)
memory usage: 70.4 KB


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [5]:
# Creating train/test values for both the mean dataset and the dropped dataset
X_train_mean, X_test_mean, y_train_mean, y_test_mean = train_test_split(X_mean, y_mean, test_size=0.3, random_state=42)
X_train_dropped, X_test_dropped, y_train_dropped, y_test_dropped = train_test_split(X_dropped, y_dropped, test_size = 0.2, random_state = 42)
print(X_train_mean.shape) ; print(X_test_mean.shape)

(1050, 6)
(450, 6)


In [6]:
# Converting y_mean
y_train_mean = to_categorical(y_train_mean)
y_test_mean = to_categorical(y_test_mean)
# Creating the neural network for mean dataset
model_mean = Sequential()
model_mean.add(Dense(500, activation='relu', input_shape=(6,)))
model_mean.add(Dense(100, activation='relu'))
model_mean.add(Dense(50, activation='relu'))
model_mean.add(Dense(2, activation='softmax'))
model_mean.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_mean.fit(X_train_mean, y_train_mean, epochs=50)

Epoch 1/50
33/33 [==============================] - 1s 3ms/step - loss: 2.5462 - accuracy: 0.5457
Epoch 2/50
33/33 [==============================] - 0s 3ms/step - loss: 0.7104 - accuracy: 0.6295
Epoch 3/50
33/33 [==============================] - 0s 2ms/step - loss: 0.6764 - accuracy: 0.6067
Epoch 4/50
33/33 [==============================] - 0s 3ms/step - loss: 0.6418 - accuracy: 0.6448
Epoch 5/50
33/33 [==============================] - 0s 2ms/step - loss: 0.7900 - accuracy: 0.6048
Epoch 6/50
33/33 [==============================] - 0s 3ms/step - loss: 0.6743 - accuracy: 0.6467
Epoch 7/50
33/33 [==============================] - 0s 2ms/step - loss: 0.7062 - accuracy: 0.6448
Epoch 8/50
33/33 [==============================] - 0s 2ms/step - loss: 0.5946 - accuracy: 0.7105
Epoch 9/50
33/33 [==============================] - 0s 2ms/step - loss: 0.5784 - accuracy: 0.6857
Epoch 10/50
33/33 [==============================] - 0s 2ms/step - loss: 0.5888 - accuracy: 0.7029
Epoch 11/50
33/33 [

In [7]:
pred_train_mean = model_mean.predict(X_train_mean)
scores_mean_train = model_mean.evaluate(X_train_mean, y_train_mean, verbose=0)
#print(scores_mean_train)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores_mean_train[1], 1 - scores_mean_train[1]))
pred_test_mean = model_mean.predict(X_test_mean)
scores_mean_test = model_mean.evaluate(X_test_mean, y_test_mean, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores_mean_test[1], 1 - scores_mean_test[1]))


33/33 [==============================] - 0s 1ms/step
Accuracy on training data: 0.9238095283508301% 
 Error on training data: 0.07619047164916992
15/15 [==============================] - 0s 2ms/step
Accuracy on test data: 0.9111111164093018% 
 Error on test data: 0.08888888359069824


In [8]:
y_train_dropped = to_categorical(y_train_dropped)
y_test_dropped = to_categorical(y_test_dropped)
# Creating the neural network for mean dataset
model_dropped = Sequential()
model_dropped.add(Dense(500, activation='relu', input_shape=(6,)))
model_dropped.add(Dense(100, activation='relu'))
model_dropped.add(Dense(50, activation='relu'))
model_dropped.add(Dense(2, activation='softmax'))
model_dropped.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_dropped.fit(X_train_dropped, y_train_dropped, epochs=50)

Epoch 1/50
37/37 [==============================] - 1s 2ms/step - loss: 1.7465 - accuracy: 0.5419
Epoch 2/50
37/37 [==============================] - 0s 2ms/step - loss: 0.8508 - accuracy: 0.5641
Epoch 3/50
37/37 [==============================] - 0s 2ms/step - loss: 0.6407 - accuracy: 0.6607
Epoch 4/50
37/37 [==============================] - 0s 2ms/step - loss: 0.7716 - accuracy: 0.5897
Epoch 5/50
37/37 [==============================] - 0s 2ms/step - loss: 0.8691 - accuracy: 0.5974
Epoch 6/50
37/37 [==============================] - 0s 2ms/step - loss: 0.6857 - accuracy: 0.6256
Epoch 7/50
37/37 [==============================] - 0s 2ms/step - loss: 0.5573 - accuracy: 0.6974
Epoch 8/50
37/37 [==============================] - 0s 2ms/step - loss: 0.5529 - accuracy: 0.7299
Epoch 9/50
37/37 [==============================] - 0s 2ms/step - loss: 0.5685 - accuracy: 0.7068
Epoch 10/50
37/37 [==============================] - 0s 2ms/step - loss: 0.6536 - accuracy: 0.6470
Epoch 11/50
37/37 [

In [9]:
pred_train_dropped = model_mean.predict(X_train_dropped)
scores_mean_dropped = model_mean.evaluate(X_train_dropped, y_train_dropped, verbose=0)

print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores_mean_dropped[1], 1 - scores_mean_dropped[1]))
pred_test_mean = model_mean.predict(X_test_dropped)
scores_dropped_test = model_mean.evaluate(X_test_dropped, y_test_dropped, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores_dropped_test[1], 1 - scores_dropped_test[1]))


37/37 [==============================] - 0s 1ms/step
Accuracy on training data: 0.9170939922332764% 
 Error on training data: 0.08290600776672363
10/10 [==============================] - 0s 2ms/step
Accuracy on test data: 0.9453924894332886% 
 Error on test data: 0.054607510566711426
